Used tutorial https://github.com/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb for reference

In [1]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T

spark = sparknlp.start()

24/11/05 14:22:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])

In [3]:
#Remove sample
data.show(5)
data = data.sample(0.001)

+-----+--------------------+
|   id|        cleaned_text|
+-----+--------------------+
|hk5r2|i had an appointm...|
|iqimz|i created this si...|
|pfzt5|hello everyone  i...|
|pk714|i grew up with bo...|
|q0q8x|i have to ask whe...|
+-----+--------------------+
only showing top 5 rows



In [4]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [5]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [6]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

In [7]:
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("normalized")\
      .setOutputCol("lemmatized")

In [8]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", 
    "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", 
    "x", "y", "z"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute']

other = ["hows", "ive", "ill", "im", "cant", "topics", "topic", "discuss", 
         "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything"]

stopwords = english + time + reddit + topic_specific + other

from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

In [9]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [10]:
pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "lemmatized")\
      .setOutputCol("pos")

In [11]:
#from sparknlp.base import Finisher

#finisher = Finisher().setInputCols(['unigrams', 'ngrams', 'pos'])

In [12]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          lemmatizer,
          stopwords_cleaner,
          ngrammer,
          pos #,
          #finisher
      ])

In [13]:
pipelineModel = my_pipeline.fit(data)
data = pipelineModel.transform(data)
data.show(1)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|            document|           tokenized|          normalized|          lemmatized|            unigrams|              ngrams|                 pos|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|2wlzcy|gtthe focus it do...|[{document, 0, 53...|[{token, 0, 4, gt...|[{token, 0, 4, gt...|[{token, 0, 4, gt...|[{token, 0, 4, gt...|[{chunk, 0, 4, gt...|[{pos, 0, 4, NN, ...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [15]:
data.columns

['id',
 'cleaned_text',
 'document',
 'tokenized',
 'normalized',
 'lemmatized',
 'unigrams',
 'ngrams',
 'pos']

In [18]:
data.select('document').show(5, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                            document|
+----------------------------------------------------------------------------------------------------+
|[{document, 0, 5304, gtthe focus it doesnt matter how many changes you try to make if none of the...|
|[{document, 0, 1477, so first off id like to say that im totally new to reddit and this is my fir...|
|[{document, 0, 7798, for some of us it is difficult to come up with ideas to excel in the current...|
|[{document, 0, 1762, hi rselfimprovement  for a while ive noticed that i really hate being the in...|
|[{document, 0, 515, title is pretty much selfexplanority  when im sober i find myself caring too ...|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [27]:
data.select('tokenized').show(1, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                           tokenized|
+----------------------------------------------------------------------------------------------------+
|[{token, 0, 4, gtthe, {sentence -> 0}, []}, {token, 6, 10, focus, {sentence -> 0}, []}, {token, 1...|
+----------------------------------------------------------------------------------------------------+
only showing top 1 row



In [30]:
data.select('normalized').show(1, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                          normalized|
+----------------------------------------------------------------------------------------------------+
|[{token, 0, 4, gtthe, {sentence -> 0}, []}, {token, 6, 10, focus, {sentence -> 0}, []}, {token, 1...|
+----------------------------------------------------------------------------------------------------+
only showing top 1 row



In [33]:
data.select('lemmatized').show(1, truncate = 1000)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [32]:
data.select('unigrams').show(1, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                            unigrams|
+----------------------------------------------------------------------------------------------------+
|[{token, 0, 4, gtthe, {sentence -> 0}, []}, {token, 15, 20, _, {sentence -> 0}, []}, {token, 33, ...|
+----------------------------------------------------------------------------------------------------+
only showing top 1 row



Create pos-tags n-grams that correspond to words n-grams

In [14]:
#Merge POS tags as just one string to be able to take it as a document in the Spark NLP Pipeline
from pyspark.sql import functions as F
pos_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
data = data.withColumn('finished_pos', pos_as_string(F.col('finished_pos')))

AnalysisException: Column 'finished_pos' does not exist. Did you mean one of the following? [pos, unigrams, cleaned_text, id, ngrams, tokenized, document, normalized, lemmatized];
'Project [id#19, cleaned_text#24, document#79, tokenized#88, normalized#95, lemmatized#103, unigrams#112, ngrams#122, pos#133, <lambda>('finished_pos)#182 AS finished_pos#183]
+- Project [id#19, cleaned_text#24, document#79, tokenized#88, normalized#95, lemmatized#103, unigrams#112, ngrams#122, UDF(array(document#79, lemmatized#103)) AS pos#133]
   +- Project [id#19, cleaned_text#24, document#79, tokenized#88, normalized#95, lemmatized#103, unigrams#112, UDF(array(lemmatized#103)) AS ngrams#122]
      +- Project [id#19, cleaned_text#24, document#79, tokenized#88, normalized#95, lemmatized#103, UDF(array(lemmatized#103)) AS unigrams#112]
         +- Project [id#19, cleaned_text#24, document#79, tokenized#88, normalized#95, UDF(array(normalized#95)) AS lemmatized#103]
            +- Project [id#19, cleaned_text

In [ ]:
data.show(3)

New pipeline for pos-tags

In [ ]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')
     
    
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams']) \

pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])

data = pos_pipeline.fit(data).transform(data)

In [ ]:
data.columns

In [ ]:
data.show(1)

In [ ]:
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS', 'VB', 'VBP', 'VBD',
                      'VBG', 'VBN', 'VNZ']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

In [ ]:
data = data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

data.show(5)

In [ ]:
def filter_pos_ngrams(finished_ngrams, finished_pos_tags):
    return [word for word, pos in zip(finished_ngrams, finished_pos_tags) 
            if (len(pos.split('_')) == 2 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS']) \
            or (len(pos.split('_')) == 3 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                  pos.split('_')[2] in ['NN', 'NNS'])]
    
udf_filter_pos_ngrams = F.udf(filter_pos_ngrams, T.ArrayType(T.StringType()))

In [ ]:
data = data.withColumn('filtered_ngrams_by_pos',
                       udf_filter_pos_ngrams(F.col('finished_ngrams'),
                                             F.col('finished_pos_ngrams')))

In [ ]:
data.show(5)

In [ ]:
data.select('id','filtered_ngrams_by_pos').show(10, truncate=100)

In [ ]:
data.select('id','filtered_unigrams_by_pos').show(10, truncate=100)

In [ ]:
data.columns

In [ ]:
data.select('cleaned_text').show(10, truncate=100)